# Test On-The-Fly Photometry for Daily Reductions

In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column
from desiutil.names import radec_to_desiname
from desiutil.log import get_logger
from desispec.io.photo import gather_tractorphot, gather_targetphot
from desitarget.targets import decode_targetid
import specprodDB.load as db
specprod = os.environ['SPECPROD']
tileid = 3867
survey, program = 'main', 'dark'

## Database connection

In [2]:
db.log = get_logger()
postgresql = db.setup_db(schema='iron', hostname='specprod-db.desi.lbl.gov', username='desi')

In [3]:
tile = db.dbSession.query(db.Tile).filter(db.Tile.tileid == tileid).one()
tile

Tile(tileid=3867)

In [4]:
assert tile.survey == survey
assert tile.program == program

In [5]:
len(tile.targets)

20183

In [6]:
def no_sky(catalog):
    """Identify objects in `catalog` that are not sky targets.
    
    Parameters
    ----------
    catalog : :class:`~astropy.table.Table`
        Any Table containing a ``TARGETID`` column.
    
    Returns
    -------
    :class:`numpy.ndarray`
        The indexes of rows that are not sky targets.
    """
    _, _, _, _, sky, _ = decode_targetid(catalog['TARGETID'])
    return np.where((sky == 0) & (catalog['TARGETID'] > 0))[0]

## Load a random tile

In [7]:
fiberassign = os.path.join(os.environ['DESI_ROOT'], 'target', 'fiberassign', 'tiles', 'trunk', f'{tileid:06d}'[0:3], f'fiberassign-{tileid:06d}.fits.gz')
with fits.open(fiberassign) as hdulist:
    hdulist.info()
    observed_targets = hdulist['FIBERASSIGN'].data
    potential_targets = hdulist['TARGETS'].data

no_sky_targets = no_sky(observed_targets)
potential_no_sky_targets = no_sky(potential_targets)
observed_cat = Table()
observed_cat['TARGETID'] = observed_targets['TARGETID'][no_sky_targets]
observed_cat['TILEID'] = tileid
observed_cat['TARGET_RA'] = observed_targets['TARGET_RA'][no_sky_targets]
observed_cat['TARGET_DEC'] = observed_targets['TARGET_DEC'][no_sky_targets]
observed_cat['PETAL_LOC'] = observed_targets['PETAL_LOC'][no_sky_targets]
observed_cat['SURVEY'] = survey
observed_cat['PROGRAM'] = program
potential_cat = Table()
potential_cat['TARGETID'] = potential_targets['TARGETID'][potential_no_sky_targets]
potential_cat['TILEID'] = tileid
potential_cat['TARGET_RA'] = potential_targets['RA'][potential_no_sky_targets]
potential_cat['TARGET_DEC'] = potential_targets['DEC'][potential_no_sky_targets]
# potential_cat['PETAL_LOC'] = potential_targets['PETAL_LOC'][potential_no_sky_targets]
potential_cat['SURVEY'] = survey
potential_cat['PROGRAM'] = program

Filename: /global/cfs/cdirs/desi/target/fiberassign/tiles/trunk/003/fiberassign-003867.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      84   ()      
  1  FIBERASSIGN    1 BinTableHDU    180   5000R x 62C   [K, I, J, J, J, J, D, D, E, E, E, E, K, B, 3A, E, E, J, D, J, I, 8A, J, J, 4A, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, E, E, E, E, K, 2A, E, E, E, E, 1A, K, K, K, K, K, K, D, D]   
  2  SKY_MONITOR    1 BinTableHDU     96   20R x 20C   [J, J, K, J, J, K, B, D, D, E, E, 8A, J, I, J, J, D, E, E, E]   
  3  GFA_TARGETS    1 BinTableHDU    136   1199R x 40C   [J, K, J, J, D, D, E, E, 4A, I, E, E, E, E, E, E, K, 2A, E, E, E, E, E, E, E, E, E, E, E, E, E, E, K, E, J, K, I, I, I, I]   
  4  TARGETS       1 BinTableHDU     80   147138R x 12C   [K, D, D, K, B, J, D, J, K, K, K, K]   
  5  POTENTIAL_ASSIGNMENTS    1 BinTableHDU     62   163875R x 3C   [K, J, J]   


In [8]:
observed_cat

TARGETID,TILEID,TARGET_RA,TARGET_DEC,PETAL_LOC,SURVEY,PROGRAM
int64,int64,float64,float64,int16,str4,str4
39633471919883859,3867,129.99976401164204,67.93041656172511,0,main,dark
39633471919884932,3867,130.22591418527318,67.97878982125872,0,main,dark
39633471924077291,3867,130.50170465408198,68.00578763078767,0,main,dark
39633474197391529,3867,130.87601892009982,68.18891252671033,0,main,dark
39633471924076786,3867,130.40706892716298,68.09295914303961,0,main,dark
39633471924078031,3867,130.63535350020834,68.0144694530991,0,main,dark
39633474197390347,3867,130.66938913791446,68.16071617122073,0,main,dark
39633471919884249,3867,130.09206414868825,67.92356821660712,0,main,dark
39633471919885170,3867,130.27900632747625,67.92075690050822,0,main,dark


In [9]:
potential_cat

TARGETID,TILEID,TARGET_RA,TARGET_DEC,SURVEY,PROGRAM
int64,int64,float64,float64,str4,str4
2404005925879810,3867,133.4399101445135,67.82437322108153,main,dark
2404005925879811,3867,133.44185934546013,67.82823149466229,main,dark
2404005925879812,3867,133.27805902114852,67.83948363786199,main,dark
2404005925879813,3867,133.28847089528776,67.84041846265472,main,dark
2404008190803968,3867,128.18399465466373,68.06496622501886,main,dark
2404008190803969,3867,128.19164716436438,68.06950585763119,main,dark
2404008199192577,3867,129.52376427694398,68.06131962761214,main,dark
2404008199192578,3867,129.5248554721843,68.05751118313499,main,dark
2404008207581184,3867,130.90646227549064,67.98937459790655,main,dark


## Tractor photometry for observed targets

In [10]:
tractorphot = gather_tractorphot(observed_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')

In [11]:
tractorphot

RELEASE,BRICKID,BRICKNAME,OBJID,BRICK_PRIMARY,MASKBITS,FITBITS,TYPE,RA,DEC,RA_IVAR,DEC_IVAR,BX,BY,DCHISQ,EBV,MJD_MIN,MJD_MAX,REF_CAT,REF_ID,PMRA,PMDEC,PARALLAX,PMRA_IVAR,PMDEC_IVAR,PARALLAX_IVAR,REF_EPOCH,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_G_N_OBS,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_N_OBS,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_N_OBS,GAIA_PHOT_VARIABLE_FLAG,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_ASTROMETRIC_EXCESS_NOISE_SIG,GAIA_ASTROMETRIC_N_OBS_AL,GAIA_ASTROMETRIC_N_GOOD_OBS_AL,GAIA_ASTROMETRIC_WEIGHT_AL,GAIA_DUPLICATED_SOURCE,GAIA_A_G_VAL,GAIA_E_BP_MIN_RP_VAL,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,APFLUX_G,APFLUX_R,APFLUX_Z,APFLUX_RESID_G,APFLUX_RESID_R,APFLUX_RESID_Z,APFLUX_BLOBRESID_G,APFLUX_BLOBRESID_R,APFLUX_BLOBRESID_Z,APFLUX_IVAR_G,APFLUX_IVAR_R,APFLUX_IVAR_Z,APFLUX_MASKED_G,APFLUX_MASKED_R,APFLUX_MASKED_Z,APFLUX_W1,APFLUX_W2,APFLUX_W3,APFLUX_W4,APFLUX_RESID_W1,APFLUX_RESID_W2,APFLUX_RESID_W3,APFLUX_RESID_W4,APFLUX_IVAR_W1,APFLUX_IVAR_W2,APFLUX_IVAR_W3,APFLUX_IVAR_W4,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,NOBS_G,NOBS_R,NOBS_Z,NOBS_W1,NOBS_W2,NOBS_W3,NOBS_W4,RCHISQ_G,RCHISQ_R,RCHISQ_Z,RCHISQ_W1,RCHISQ_W2,RCHISQ_W3,RCHISQ_W4,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACFLUX_W1,FRACFLUX_W2,FRACFLUX_W3,FRACFLUX_W4,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,ANYMASK_G,ANYMASK_R,ANYMASK_Z,ALLMASK_G,ALLMASK_R,ALLMASK_Z,WISEMASK_W1,WISEMASK_W2,PSFSIZE_G,PSFSIZE_R,PSFSIZE_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,NEA_G,NEA_R,NEA_Z,BLOB_NEA_G,BLOB_NEA_R,BLOB_NEA_Z,PSFDEPTH_W1,PSFDEPTH_W2,PSFDEPTH_W3,PSFDEPTH_W4,WISE_COADD_ID,WISE_X,WISE_Y,LC_FLUX_W1,LC_FLUX_W2,LC_FLUX_IVAR_W1,LC_FLUX_IVAR_W2,LC_NOBS_W1,LC_NOBS_W2,LC_FRACFLUX_W1,LC_FRACFLUX_W2,LC_RCHISQ_W1,LC_RCHISQ_W2,LC_MJD_W1,LC_MJD_W2,LC_EPOCH_INDEX_W1,LC_EPOCH_INDEX_W2,SERSIC,SERSIC_IVAR,SHAPE_R,SHAPE_R_IVAR,SHAPE_E1,SHAPE_E1_IVAR,SHAPE_E2,SHAPE_E2_IVAR,LS_ID,TARGETID
int16,int32,str8,int32,bool,int16,int16,str3,float64,float64,float32,float32,float32,float32,float32[5],float32,float64,float64,str2,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,int16,float32,float32,int16,bool,float32,float32,int16,int16,float32,bool,float32,float32,float32,float32,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,uint8,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str8,float32,float32,float32[15],float32[15],float32[15],float32[15],int16[15],int16[15],float32[15],float32[15],float32[15],float32[15],float64[15],float64[15],int16[15],int16[15],float32,float32,float32,float32,float32,float32,float32,float32,int64,int64
9011,637724,1300p680,1619,True,0,0,REX,129.99976401164204,67.93041656172511,42750837000.0,43095240000.0,1990.5249,843.44836,1955.9313 .. 0.

## Target information for observed targets

In [12]:
targetphot = gather_targetphot(observed_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')
targetphot['SURVEY'] = observed_cat['SURVEY']
targetphot['PROGRAM'] = observed_cat['PROGRAM']
targetphot['TILEID'] = observed_cat['TILEID']
targetphot['DESINAME'] = radec_to_desiname(targetphot['RA'], targetphot['DEC'])
inan = np.logical_or(np.isnan(targetphot['PMRA']), np.isnan(targetphot['PMDEC']))
if np.any(inan):
    targetphot['PMRA'][inan] = 0.0
    targetphot['PMDEC'][inan] = 0.0
targetphot

RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,RA,RA_IVAR,DEC,DEC_IVAR,DCHISQ,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,REF_EPOCH,WISEMASK_W1,WISEMASK_W2,MASKBITS,LC_FLUX_W1,LC_FLUX_W2,LC_FLUX_IVAR_W1,LC_FLUX_IVAR_W2,LC_NOBS_W1,LC_NOBS_W2,LC_MJD_W1,LC_MJD_W2,SHAPE_R,SHAPE_E1,SHAPE_E2,SHAPE_R_IVAR,SHAPE_E1_IVAR,SHAPE_E2_IVAR,SERSIC,SERSIC_IVAR,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PMRA,PMRA_IVAR,PMDEC,PMDEC_IVAR,PHOTSYS,TARGETID,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SCND_TARGET,SV1_SCND_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SURVEY,PROGRAM,TILEID,DESINAME
int16,int32,str8,int32,str4,float64,float32,float64,float32,float32[5],float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,float32[15],float32[15],float32[15],float32[15],int16[15],int16[15],float64[15],float64[15],float32,float32,float32,float32,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,uint8,float32,float32,float32,float32,float32,float32,str1,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,str4,str4,int64,str22
9011,637724,1300p680,1619,REX,129.99976401164204,42750837000.0,67.93041656172511,43095240000.0,1955.9313 .. 0.0,0.03723232,0.15403047,0.75954026,5.739367,256.32214,123.59633,61.201317,0.8956413,0.92844635,0.9593226,0.0,0.0,0.0,0.024760377,0.039331,0.010728815,0.98911744,0.9818307,0.9965942,2,3,3,329.56912,156.20256,104.29852,240.36511,117.59248,54.648235,29.839472,21.946272,-48.227264,-536.0235,3.2750506,0.9210226,0.00040354583,5.0037156e-06,0.9937101,0.9961325,0.9991739,0.99968797,0,0,0,0.09424214,0.46471778,3.5115793,0.09424214,0.46471778,3.5115793,0.0,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.35319605,0.0,0.0,3445.7798,0.0,0.0,1.0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,N,39633471919883859,0.9638007113419483,1,3200,2,7773,0,257,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,main,dark,3867,DESI J129.9997+67.9304
9011,637724,1300p680,2692,REX,130.22591418527318,5037999000.0,67.97878982125872,5227806000.0,373.0811 .. 0.0,0.03711949,0.7750379,0.9475041,1.4774636,248.69882,118.07855,61.37548,0.89594054,0.92865527,0.95944333,0.0,0.0,0.0,0.026685862,0.09917747,0.010309853,0.987,0.9810207,0.99114937,2,3,3,329.56912,156.20256,106.6778,240.36511,117.59248,55.74153,6.578294,2.6278782,-67.02996,-441.8467,4.016817,1.0150393,0.0004932014,6.5734534e-06,0.9937291,0.9961442,0.9991764,0.9996889,0,0,0,0.46204898,0.56486696,0.8808092,0.46204898,0.56486696,0.8808092,0.0,0,0,0,0.0 ..

## Tractor photometry for potential targets

In [13]:
potential_tractorphot = gather_tractorphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')

In [14]:
potential_tractorphot

RELEASE,BRICKID,BRICKNAME,OBJID,BRICK_PRIMARY,MASKBITS,FITBITS,TYPE,RA,DEC,RA_IVAR,DEC_IVAR,BX,BY,DCHISQ,EBV,MJD_MIN,MJD_MAX,REF_CAT,REF_ID,PMRA,PMDEC,PARALLAX,PMRA_IVAR,PMDEC_IVAR,PARALLAX_IVAR,REF_EPOCH,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_G_N_OBS,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_N_OBS,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_N_OBS,GAIA_PHOT_VARIABLE_FLAG,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_ASTROMETRIC_EXCESS_NOISE_SIG,GAIA_ASTROMETRIC_N_OBS_AL,GAIA_ASTROMETRIC_N_GOOD_OBS_AL,GAIA_ASTROMETRIC_WEIGHT_AL,GAIA_DUPLICATED_SOURCE,GAIA_A_G_VAL,GAIA_E_BP_MIN_RP_VAL,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,APFLUX_G,APFLUX_R,APFLUX_Z,APFLUX_RESID_G,APFLUX_RESID_R,APFLUX_RESID_Z,APFLUX_BLOBRESID_G,APFLUX_BLOBRESID_R,APFLUX_BLOBRESID_Z,APFLUX_IVAR_G,APFLUX_IVAR_R,APFLUX_IVAR_Z,APFLUX_MASKED_G,APFLUX_MASKED_R,APFLUX_MASKED_Z,APFLUX_W1,APFLUX_W2,APFLUX_W3,APFLUX_W4,APFLUX_RESID_W1,APFLUX_RESID_W2,APFLUX_RESID_W3,APFLUX_RESID_W4,APFLUX_IVAR_W1,APFLUX_IVAR_W2,APFLUX_IVAR_W3,APFLUX_IVAR_W4,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,NOBS_G,NOBS_R,NOBS_Z,NOBS_W1,NOBS_W2,NOBS_W3,NOBS_W4,RCHISQ_G,RCHISQ_R,RCHISQ_Z,RCHISQ_W1,RCHISQ_W2,RCHISQ_W3,RCHISQ_W4,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACFLUX_W1,FRACFLUX_W2,FRACFLUX_W3,FRACFLUX_W4,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,ANYMASK_G,ANYMASK_R,ANYMASK_Z,ALLMASK_G,ALLMASK_R,ALLMASK_Z,WISEMASK_W1,WISEMASK_W2,PSFSIZE_G,PSFSIZE_R,PSFSIZE_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,NEA_G,NEA_R,NEA_Z,BLOB_NEA_G,BLOB_NEA_R,BLOB_NEA_Z,PSFDEPTH_W1,PSFDEPTH_W2,PSFDEPTH_W3,PSFDEPTH_W4,WISE_COADD_ID,WISE_X,WISE_Y,LC_FLUX_W1,LC_FLUX_W2,LC_FLUX_IVAR_W1,LC_FLUX_IVAR_W2,LC_NOBS_W1,LC_NOBS_W2,LC_FRACFLUX_W1,LC_FRACFLUX_W2,LC_RCHISQ_W1,LC_RCHISQ_W2,LC_MJD_W1,LC_MJD_W2,LC_EPOCH_INDEX_W1,LC_EPOCH_INDEX_W2,SERSIC,SERSIC_IVAR,SHAPE_R,SHAPE_R_IVAR,SHAPE_E1,SHAPE_E1_IVAR,SHAPE_E2,SHAPE_E2_IVAR,LS_ID,TARGETID
int16,int32,str8,int32,bool,int16,int16,str3,float64,float64,float32,float32,float32,float32,float32[5],float32,float64,float64,str2,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,int16,float32,float32,int16,bool,float32,float32,int16,int16,float32,bool,float32,float32,float32,float32,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,uint8,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str8,float32,float32,float32[15],float32[15],float32[15],float32[15],int16[15],int16[15],float32[15],float32[15],float32[15],float32[15],float64[15],float64[15],int16[15],int16[15],float32,float32,float32,float32,float32,float32,float32,float32,int64,int64
0,0,,0,False,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0 .. 0.0,0.0,0.0,0.0,,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,False,0.0,0.0

## Target information for potential targets

In [15]:
potential_targetphot = gather_targetphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')
potential_targetphot['SURVEY'] = potential_cat['SURVEY']
potential_targetphot['PROGRAM'] = potential_cat['PROGRAM']
potential_targetphot['TILEID'] = potential_cat['TILEID']
potential_targetphot['DESINAME'] = radec_to_desiname(potential_targetphot['RA'], potential_targetphot['DEC'])
inan = np.logical_or(np.isnan(potential_targetphot['PMRA']), np.isnan(potential_targetphot['PMDEC']))
if np.any(inan):
    potential_targetphot['PMRA'][inan] = 0.0
    potential_targetphot['PMDEC'][inan] = 0.0
potential_targetphot

RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,RA,RA_IVAR,DEC,DEC_IVAR,DCHISQ,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,REF_EPOCH,WISEMASK_W1,WISEMASK_W2,MASKBITS,LC_FLUX_W1,LC_FLUX_W2,LC_FLUX_IVAR_W1,LC_FLUX_IVAR_W2,LC_NOBS_W1,LC_NOBS_W2,LC_MJD_W1,LC_MJD_W2,SHAPE_R,SHAPE_E1,SHAPE_E2,SHAPE_R_IVAR,SHAPE_E1_IVAR,SHAPE_E2_IVAR,SERSIC,SERSIC_IVAR,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PMRA,PMRA_IVAR,PMDEC,PMDEC_IVAR,PHOTSYS,TARGETID,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SCND_TARGET,SV1_SCND_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SURVEY,PROGRAM,TILEID,DESINAME
int16,int32,str8,int32,str4,float64,float32,float64,float32,float32[5],float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,float32[15],float32[15],float32[15],float32[15],int16[15],int16[15],float64[15],float64[15],float32,float32,float32,float32,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,uint8,float32,float32,float32,float32,float32,float32,str1,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,str4,str4,int64,str22
0,0,,0,,133.4399101445135,0.0,67.82437322108153,0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2015.5,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,,2404005925879810,0.1729051562495022,1,1700,5,0,0,4611686018427387904,0,0,0,0,0,0,0,0,0,0,0,70368744177664,0,0,0,main,dark,3867,DESI J133.4399+67.8243
0,0,,0,,133.44185934546013,0.0,67.82823149466229,0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2015.5,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,,2404005925879811,0.49123348190990845,1,1700,5,0,0,4611686018427387904,0,0,0,0,0,0,0,0,0,0,0,70368744177664,0,0,0,main,dark,3867,DESI J133.4418+67.8282
0,0,,0,,133.27805902114852,0.0,67.83948363786199,0.0,0.0 .. 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2015.5,0,0,0,

## Compare "gather" results to the database

In [16]:
assert len(potential_targetphot) == len(tile.targets)

In [17]:
for t in tile.targets:
    p = t.photometry
    i = np.where(potential_targetphot['TARGETID'] == t.targetid)[0]
    j = np.where(potential_tractorphot['TARGETID'] == t.targetid)[0]
    row = potential_targetphot[i]
    phot_row = potential_tractorphot[i]
    assert t.tileid == tileid
    assert t.survey == survey
    assert t.program == program
    for col in ('DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'SCND_TARGET'):
        assert row[col] == getattr(t, col.lower())
    for col in ('BRICKID', 'BRICKNAME', 'TYPE', 'MASKBITS'):
        if col == 'TYPE':
            assert phot_row[col] == getattr(p, 'morphtype')
        else:
            assert phot_row[col] == getattr(p, col.lower())